In [1]:
import os
import rasterio
import numpy as np

# Folder path containing the TIFF files
folder_path = r"G:\Hangkai\Anttarctic Vegetation Dynamic\Snow"

# File names for the TIFF files from 2002 to 2023
tif_files = [f"Good_MinSnowCover_{year}.tif" for year in range(2002, 2024)]

# Loop over the files
for tif_file in tif_files:
    file_path = os.path.join(folder_path, tif_file)
    
    # Open the TIFF file
    with rasterio.open(file_path) as dataset:
        # Read the data into a numpy array
        data = dataset.read(1)

        # Mask out the values where data equals 111 (considered NaN)
        valid_data = np.where(data != 111, data, np.nan)

        # Calculate the area of pixels with values <= 30
        pixel_size = dataset.res[0] * dataset.res[1]  # Calculate the area of each pixel
        area_pixels = np.count_nonzero(valid_data <= 30)
        total_area = area_pixels * pixel_size  # Total area with value <= 30

        # Calculate the mean value, ignoring NaN values
        mean_value = np.nanmean(valid_data)

        # Print the results
        print(f"File: {tif_file}")
        print(f"Total area with value <= 30: {total_area:.2f} square meters")
        print(f"Mean value (ignoring NaN): {mean_value:.2f}\n")

File: Good_MinSnowCover_2002.tif
Total area with value <= 30: 705279750000.00 square meters
Mean value (ignoring NaN): 47.76

File: Good_MinSnowCover_2003.tif
Total area with value <= 30: 623852500000.00 square meters
Mean value (ignoring NaN): 48.32

File: Good_MinSnowCover_2004.tif
Total area with value <= 30: 732776000000.00 square meters
Mean value (ignoring NaN): 48.28

File: Good_MinSnowCover_2005.tif
Total area with value <= 30: 792706500000.00 square meters
Mean value (ignoring NaN): 46.59

File: Good_MinSnowCover_2006.tif
Total area with value <= 30: 775881750000.00 square meters
Mean value (ignoring NaN): 46.99

File: Good_MinSnowCover_2007.tif
Total area with value <= 30: 771018000000.00 square meters
Mean value (ignoring NaN): 46.18

File: Good_MinSnowCover_2008.tif
Total area with value <= 30: 719546500000.00 square meters
Mean value (ignoring NaN): 47.66

File: Good_MinSnowCover_2009.tif
Total area with value <= 30: 651485750000.00 square meters
Mean value (ignoring NaN):

In [17]:
import geopandas as gpd
import os

# Define the folder and file naming pattern
folder_path = 'G:/Hangkai/Anttarctic Vegetation Dynamic/Snow/'
file_pattern = 'Zonal_Zonal_{}.shp'

# Initialize a nested dictionary to store totals per year and per subregion
subregion_totals_by_year = {}  # {subregion: {year: total}}

# Loop through each year from 2002 to 2023
for year in range(2002, 2024):
    file_path = os.path.join(folder_path, file_pattern.format(year))

    # Read the shapefile
    print(f"Reading shapefile for year {year}...")
    try:
        gdf = gpd.read_file(file_path)
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        continue

    # Initialize the 'HISTO_SUM' column for this year
    gdf['HISTO_SUM'] = 0

    # Sum the values from HISTO_0 to HISTO_30 (if they exist)
    for i in range(31):
        column_name = f'HISTO_{i}'
        if column_name in gdf.columns:
            gdf['HISTO_SUM'] += gdf[column_name].fillna(0)*0.25

    # Sum the 'HISTO_SUM' per subregion (using 'Subregions' column)
    subregion_sums = gdf.groupby('Subregions')['HISTO_SUM'].sum()

    # Store the subregion sums for this year
    for subregion, value in subregion_sums.items():
        if subregion not in subregion_totals_by_year:
            subregion_totals_by_year[subregion] = {}
        subregion_totals_by_year[subregion][year] = value

# Display the results
print("\nTotal valid pixels per subregion for each year (HISTO_0 to HISTO_30):")
for subregion, year_data in subregion_totals_by_year.items():
    print(f"\nSubregion: {subregion}")
    for year, total in sorted(year_data.items()):
        print(f"{total}")

# Optional: Calculate and display the grand total
grand_total = sum(
    sum(year_data.values()) for year_data in subregion_totals_by_year.values()
)
print(f"\nGrand Total (All Subregions, 2002-2023): {grand_total}")

Reading shapefile for year 2002...
Reading shapefile for year 2003...
Reading shapefile for year 2004...
Reading shapefile for year 2005...
Reading shapefile for year 2006...
Reading shapefile for year 2007...
Reading shapefile for year 2008...
Reading shapefile for year 2009...
Reading shapefile for year 2010...
Reading shapefile for year 2011...
Reading shapefile for year 2012...
Reading shapefile for year 2013...
Reading shapefile for year 2014...
Reading shapefile for year 2015...
Reading shapefile for year 2016...
Reading shapefile for year 2017...
Reading shapefile for year 2018...
Reading shapefile for year 2019...
Reading shapefile for year 2020...
Reading shapefile for year 2021...
Reading shapefile for year 2022...
Reading shapefile for year 2023...

Total valid pixels per subregion for each year (HISTO_0 to HISTO_30):

Subregion: A-Ap
32962.0
55522.25
36754.0
16544.0
23999.75
38463.5
53342.25
60905.5
64958.0
51718.25
42640.5
56912.25
46662.5
34697.25
66328.75
65260.5
64833.5